In [ ]:
import json, re
import numpy as np
import pandas as pd
import tifffile as tiff
import qupath_utils as qutils 

# ----------- user inputs -----------
tif_path     = r"C:\Users\Jenny\Desktop\JJ004\Raw\slide1_slice5.tif" # 16 bit image 
geojson_path = r"C:\Users\Jenny\Desktop\JJ004\Qupath\slide1_slice5.geojson" # Exported from aligned ABBA 

num_idx, den_idx = 3,2       # ch3, ch4 (0-based)
eps = 1e-9 # small num to prevent inf 

# ----------- image -----------
img = tiff.imread(tif_path)
if img.ndim == 2: img = img[np.newaxis, ...]
elif img.ndim == 3 and img.shape[-1] <= 8: img = np.moveaxis(img, -1, 0)
C, H, W = img.shape

num = img[num_idx].astype(np.float32, copy=False)
den = img[den_idx].astype(np.float32, copy=False)
ratio_pix = num / (num + den + eps)

# ----------- load geojson & relationships -----------
gj = json.load(open(geojson_path, "r"))
features = [f for f in gj.get("features", [])
            if isinstance(f, dict) and (f.get("geometry") or {}).get("type") in ("Polygon","MultiPolygon")]

id_to_feat, parent_to_children, id_to_name = {}, {}, {}
rows_meta = []
for f in features:
    props = f.get("properties") or {}
    meas  = props.get("measurements") or {}
    oid = meas.get("ID")
    pid = meas.get("Parent ID")
    area, side = qutils.parse_area_side(props)
    base = qutils.normalize_name(area)
    # name map from ALL features
    if oid is not None:
        id_to_feat[oid] = f
        id_to_name[oid] = base
    if pid is not None:
        parent_to_children.setdefault(pid, []).append(oid)
    rows_meta.append({"orig_id": oid, "parent_id": pid, "roi_name": base, "side": side})

meta_df = pd.DataFrame(rows_meta)
meta_df["orig_id"]   = meta_df["orig_id"].astype("Int64")
meta_df["parent_id"] = meta_df["parent_id"].astype("Int64")

all_ids = set(id_to_feat.keys())
parent_ids = set(parent_to_children.keys())
leaf_ids = all_ids - parent_ids

# ----------- measure EVERY feature independently -----------
# level = 'leaf' if ID in leaf_ids; 'parent' if ID in parent_ids; 'orphan' if no ID
records = []

for f in features:
    props = f.get("properties") or {}
    meas  = props.get("measurements") or {}
    oid   = meas.get("ID")
    pid   = meas.get("Parent ID")
    area, side = qutils.parse_area_side(props)
    name = qutils.normalize_name(area)
    level = ("orphan" if oid is None else ("leaf" if oid in leaf_ids else "parent"))
    parent_name = id_to_name.get(pid)

    g = qutils.fix_clip(f["geometry"], H=H, W=W)
    if g is None:
        records.append({
            "level": level, "roi_name": name, "side": side, "parent_name": parent_name,
            "orig_id": oid, "parent_id": pid,
            "n_pixels": 0, "sum_num": 0.0, "sum_den": 0.0,
            "mean_pixelwise_ratio": np.nan, "ratio_of_sums": np.nan
        })
        continue

    mask = qutils.mask_from_geom(g, H, W)
    n, sN, sD, mean_ratio, ros = qutils.summarize_mask(mask, num, den, eps)
    records.append({
        "level": level, "roi_name": name, "side": side, "parent_name": parent_name,
        "orig_id": oid, "parent_id": pid,
        "n_pixels": n, "sum_num": sN, "sum_den": sD,
        "mean_pixelwise_ratio": mean_ratio, "ratio_of_sums": ros
    })


roi_df = pd.DataFrame.from_records(records)

# ----------- tidy -----------

# Exclusions
roi_df = qutils.drop_branches(roi_df, superparent_label="fiber tracts", recursive=True)
roi_df = qutils.drop_branches(roi_df, superparent_label= "VS", recursive=True)

roi_df = roi_df[~(roi_df["roi_name"]=="Root")]
roi_df = roi_df.sort_values(["level","parent_name","roi_name","side"]).reset_index(drop=True)

print(roi_df.head())
print(roi_df["level"].value_counts(dropna=False))
